<a href="https://colab.research.google.com/github/mdkennedy3/Useful_Robotics_Scripts/blob/master/DenseTact_camera_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DenseTact Testing Colab

This page allows one to quickly see the raw image inside a USB connected DenseTact sensor
![](https://arm.stanford.edu/sites/g/files/sbiybj21131/files/styles/card_1900x950/public/media/image/integrated_densetact-min_1.png?h=6ad01adc&itok=BNDEBePM)

## First we ask for access to your camera:

In [22]:
from IPython.display import display, Javascript
from google.colab.output import eval_js

# Function to prompt for camera access
def request_camera_access():
    js = Javascript('''
    (async function() {
        try {
            // This line prompts the user for camera access
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            // Use the stream or just trigger the prompt
            stream.getTracks().forEach(track => track.stop()); // Stops the camera after access is granted
            return true; // Indicates success
        } catch (error) {
            return {error: error.message}; // Error handling
        }
    })()
    ''')
    display(js)
    return eval_js('''
    (async function() {
        try {
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            stream.getTracks().forEach(track => track.stop());
            return true;
        } catch (error) {
            return {error: error.message};
        }
    })()
    ''')

# Request camera access as soon as this cell runs
camera_access = request_camera_access()
if isinstance(camera_access, dict) and 'error' in camera_access:
    print("Failed to access camera:", camera_access['error'])
else:
    print("Camera access granted.")

# Further code to utilize the camera can follow here


<IPython.core.display.Javascript object>

Camera access granted.


We then list all the cameras and their IDs available, you will need to copy the specific ID you want to use in the next block

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js

def get_video_input_devices():
    js = Javascript('''
    (async function() {
        const devices = await navigator.mediaDevices.enumerateDevices();
        const videoDevices = devices.filter(device => device.kind === 'videoinput');
        return videoDevices.map(device => ({label: device.label, id: device.deviceId}));
    })()
    ''')
    display(js)
    try:
        # Directly executing the anonymous async function
        result = eval_js('(async function() { \
            const devices = await navigator.mediaDevices.enumerateDevices(); \
            const videoDevices = devices.filter(device => device.kind === "videoinput"); \
            return videoDevices.map(device => ({label: device.label, id: device.deviceId})); \
        })()')
        return result
    except Exception as e:
        print(f"Failed to execute JS: {e}")
        return []

# Display the list of video input devices
devices = get_video_input_devices()
if devices:
    print("Available video input devices:")
    for device in devices:
        print(f"Label: {device['label']}, ID: {device['id']}")
else:
    print("No devices found or JavaScript execution failed.")


Given the USB ID (not webcam), copy that into the auto-generated prompt below

In [23]:
from IPython.display import display, Javascript
from google.colab.output import eval_js

# Function to fetch video input devices using JavaScript directly
def get_video_input_devices():
    js = Javascript('''
    (async function() {
        const devices = await navigator.mediaDevices.enumerateDevices();
        const videoDevices = devices.filter(device => device.kind === 'videoinput');
        return videoDevices.map(device => ({label: device.label, id: device.deviceId}));
    })()
    ''')
    display(js)
    return eval_js('''
        (async function() {
            const devices = await navigator.mediaDevices.enumerateDevices();
            const videoDevices = devices.filter(device => device.kind === 'videoinput');
            return videoDevices.map(device => ({label: device.label, id: device.deviceId}));
        })()
    ''')

# Function to create a video element and stream video from a specified camera
def video_stream(device_id):
    js = Javascript(f'''
    (async function() {{
        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({{video: {{deviceId: {{exact: "{device_id}"}}}}}});
        document.body.appendChild(video);
        video.srcObject = stream;
        await video.play();
    }})()
    ''')
    display(js)

# Main execution flow
devices = get_video_input_devices()
print("Available video input devices:")
for device in devices:
    print(f"Label: {device['label']}, ID: {device['id']}")

# Prompt for manual input of the desired camera device ID
device_id = input("Enter the Device ID of the USB camera you want to use: ")

# Stream video from the specified camera
video_stream(device_id)


<IPython.core.display.Javascript object>

Available video input devices:
Label: FaceTime HD Camera (467C:1317), ID: 27ffefd278570b5bfff3cef24e0e73604624bfc9d90b35939616ad95f2e0623a
Label: P600 (1b17:0522), ID: f3d484f213d15e9a2306612805e5c0750a133468dce82ffb0b11bc80b96fd379
Enter the Device ID of the USB camera you want to use: f3d484f213d15e9a2306612805e5c0750a133468dce82ffb0b11bc80b96fd379


<IPython.core.display.Javascript object>

# Vary Image Brightness
This version allows one to use a sliding bar to update image brightness

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
import ipywidgets as widgets

# Function to fetch video input devices using JavaScript directly
def get_video_input_devices():
    js = Javascript('''
    (async function() {
        const devices = await navigator.mediaDevices.enumerateDevices();
        const videoDevices = devices.filter(device => device.kind === 'videoinput');
        return videoDevices.map(device => ({label: device.label, id: device.deviceId}));
    })()
    ''')
    display(js)
    return eval_js('''
        (async function() {
            const devices = await navigator.mediaDevices.enumerateDevices();
            const videoDevices = devices.filter(device => device.kind === 'videoinput');
            return videoDevices.map(device => ({label: device.label, id: device.deviceId}));
        })()
    ''')

# Function to create a video element and stream video from a specified camera with adjustable brightness
def video_stream(device_id):
    js = Javascript(f'''
    (async function() {{
        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({{video: {{deviceId: {{exact: "{device_id}"}}}}}});
        document.body.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Create a slider to control the brightness
        const slider = document.createElement('input');
        slider.type = 'range';
        slider.min = 0;
        slider.max = 2;
        slider.step = 0.01;
        slider.value = 1;
        slider.style.width = '640px';
        slider.style.marginTop = '10px';
        document.body.appendChild(slider);

        // Adjust brightness based on slider value
        slider.oninput = function() {{
            video.style.filter = 'brightness(' + slider.value + ')';
        }};
    }})()
    ''')
    display(js)

# Main execution flow
devices = get_video_input_devices()
print("Available video input devices:")
for device in devices:
    print(f"Label: {device['label']}, ID: {device['id']}")

# Prompt for manual input of the desired camera device ID
device_id = input("Enter the Device ID of the USB camera you want to use: ")

# Stream video from the specified camera
video_stream(device_id)